In [ ]:
# Install the necessary libraries
!pip install google-cloud-vision pillow google-colab

import io
import os
from google.cloud import vision
from google.colab import files
from PIL import Image, ImageDraw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.6/459.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.1 MB/s eta 0:00:00


In [ ]:
# Upload your service account key JSON file
print("Please upload your Google Cloud service account key JSON file:")
uploaded = files.upload()
service_account_key = list(uploaded.keys())[0]

# Set up the Vision API client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key
client = vision.ImageAnnotatorClient()

# Function to perform text detection on an uploaded image
def detect_text(path):
    """Detects text in the file."""
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    if response.error.message:
        raise Exception(
            f'{response.error.message}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'
        )

    return texts

Please upload your Google Cloud service account key JSON file:


Saving WeissTLServiceKey.json to WeissTLServiceKey.json


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("EricZ0u/WeissTranslate")
model = AutoModelForSeq2SeqLM.from_pretrained("EricZ0u/WeissTranslate")
client = vision.ImageAnnotatorClient()

def detect_text(path):
    """Detects text in the file."""
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    if response.error.message:
        raise Exception(
            f'{response.error.message}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'
        )

    return texts

In [ ]:
print("Please upload an image file:")
uploaded_image = files.upload()
image_path = list(uploaded_image.keys())[0]

# Perform text detection
texts = detect_text(image_path)

# Display the original image
image = Image.open(image_path)
draw = ImageDraw.Draw(image)

# Annotate image with detected text
for text in texts:
    vertices = [(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices]
    draw.polygon(vertices, outline='red')

# Display the image with annotated text
image.show()

# Print detected text
detected_text = texts[0].description if texts else ""
print("Detected text:")
print(detected_text)

Please upload an image file:


Saving Screenshot 2024-06-07 at 3.26.07 PM.png to Screenshot 2024-06-07 at 3.26.07 PM.png
Detected text:
[する]


In [ ]:
detected_text = preprocess(detected_text)
print(detected_text)
tokenizer.src_lang = "ja_XX"
encoded_ar = tokenizer(detected_text, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
translated = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(translated)
translated = postprocess(translated[0])
print("Translated text:")
print(translated)

起 [このカードを
あなたは自分のレベル1以上のキャラを1枚選び、
する]
そのターン中、パワーを+1500。
集中 [[CIRCLE_SYMBOL_1] このカードを
あなたは自分の山札の上から4枚をめく
する]
り、控え室に置く。 それらのカードのクライマックス1枚につき、あなたは自
分の控え室の〈ホロライブ》のキャラを1枚まで選び、手札に戻す。
['S [Choose 1 of your Level 1 or higher Characters this] For the turn that you gain +1500 Power. BRAINSTORM [[CIRCLE_SYMBOL_1] Flip this over from top of your Library to the Waiting Room. For each Climax card revealed this way, choose up to 1 ::Hololive:: Character in your Waiting Room and return it to your hand.']
Translated text:
S [Choose 1 of your Level 1 or higher Characters this] For the turn that you gain +1500 Power. BRAINSTORM [① Flip this over from top of your Library to the Waiting Room. For each Climax card revealed this way, choose up to 1 ::Hololive:: Character in your Waiting Room and return it to your hand.


In [ ]:
import re

def preprocess(text):
    processed_text = re.sub(r'①', '[CIRCLE_SYMBOL_1]', text)
    processed_text = re.sub(r'\[②\]', '[CIRCLE_SYMBOL_2]', processed_text)
    processed_text = re.sub(r'\[③\]', '[CIRCLE_SYMBOL_3]', processed_text)
    processed_text = re.sub(r'\[④\]', '[CIRCLE_SYMBOL_4]', processed_text)
    processed_text = re.sub(r'\[⑤\]', '[CIRCLE_SYMBOL_5]', processed_text)
    processed_text = re.sub(r'\[⑥\]', '[CIRCLE_SYMBOL_6]', processed_text)
    processed_text = re.sub(r'\[⑦\]', '[CIRCLE_SYMBOL_7]', processed_text)
    processed_text = re.sub(r'\[⑧\]', '[CIRCLE_SYMBOL_8]', processed_text)
    processed_text = re.sub(r'\[⑨\]', '[CIRCLE_SYMBOL_9]', processed_text)
    return processed_text

def postprocess(text):
    processed_text = text.replace('[CIRCLE_SYMBOL_1]', '①')
    processed_text = processed_text.replace('[CIRCLE_SYMBOL_2]', '②')
    processed_text = processed_text.replace('[CIRCLE_SYMBOL_3]', '③')
    processed_text = processed_text.replace('[CIRCLE_SYMBOL_4]', '④')
    processed_text = processed_text.replace('[CIRCLE_SYMBOL_5]', '⑤')
    processed_text = processed_text.replace('[CIRCLE_SYMBOL_6]', '⑥')
    processed_text = processed_text.replace('[CIRCLE_SYMBOL_7]', '⑦')
    processed_text = processed_text.replace('[CIRCLE_SYMBOL_8]', '⑧')
    processed_text = processed_text.replace('[CIRCLE_SYMBOL_9]', '⑨')
    return processed_text


